In [1]:
!pip install deflate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 KB 2.2 MB/s eta 0:00:00


In [2]:
!wget https://dl3.pushbulletusercontent.com/TvHPfXu81gCythGyuHcvDrvgoNgD26H5/code.zip
!unzip code.zip
!rm EC.py
!rm mnca.py

--2023-03-23 23:35:17--  https://dl3.pushbulletusercontent.com/TvHPfXu81gCythGyuHcvDrvgoNgD26H5/code.zip
Resolving dl3.pushbulletusercontent.com (dl3.pushbulletusercontent.com)... 35.208.62.122
Connecting to dl3.pushbulletusercontent.com (dl3.pushbulletusercontent.com)|35.208.62.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4933 (4.8K) [application/x-zip-compressed]
Saving to: ‘code.zip’

code.zip            100%[===================>]   4.82K  --.-KB/s    in 0.001s  

2023-03-23 23:35:17 (3.18 MB/s) - ‘code.zip’ saved [4933/4933]

Archive:  code.zip
  inflating: EC.py                   
  inflating: mnca.py                 
  inflating: neighborhoods/mask_c1.txt  
  inflating: neighborhoods/mask_c2.txt  
  inflating: neighborhoods/mask_c3.txt  


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import deflate
import os
import sys
np.set_printoptions(threshold=sys.maxsize)



def extract_neighborhood_from_file(file_name):
    with open(file_name, "r") as f:
        lines = f.readlines()
        neighborhood = []
        for i, line in enumerate(lines):
            line = line.strip()
            for j, value in enumerate(line.split(" ")):
                if value == "1":
                    neighborhood.append((i-1, j-1))
        return neighborhood



def init_board(width, height, init_state):
    if init_state == "single cell in center":
        # print("Board Initialised with, single cell in center ")
        board = np.zeros((height, width), dtype=np.int)
        board[height//2, width//2] = 1
    elif init_state == "random cells with some probability":
        # print("Board Initialised with, random cells with some probability ")
        p = 0.50 # probability of a cell being alive
        board = np.random.choice([0, 1], size=(height, width), p=[1-p, p])
    elif init_state == "random cells with 2 different states":
        # print("random cells with 2 different states")
        p1 = 0.40 # probability of a cell being state 1
        board = np.random.choice([0, 1, 2], size=(height, width), p=[1-p1, p1/2, p1/2])
    else:
        raise ValueError("Invalid initial state")
    return board

def mnca(range_of_neighborhood_sums):   

    ###############----CONSTANTS----###############
    width = 100
    height = 100
    steps = 50
    init_state = "random cells with some probability"
    # Fixed Neighbohoods
    nh1 = extract_neighborhood_from_file('neighborhoods/mask_c1.txt')
    nh2 = extract_neighborhood_from_file('neighborhoods/mask_c2.txt')
    nh3 = extract_neighborhood_from_file('neighborhoods/mask_c3.txt')

    # neighborhoods = [nh1,nh2,nh3]
    neighborhoods = [nh1,nh2,nh3]
    board_arr = []
    board = init_board(width, height, init_state)
    ###############----CONSTANTS----###############


    for eachstep in range(steps):
        new_board = np.zeros_like(board, dtype=int)
        height, width = board.shape
        for i in range(height):
            for j in range(width):
                next_state = board[i, j]
                for neighborhood, range_of_neighborhood_sum in zip(neighborhoods, range_of_neighborhood_sums):
                    cell_neighborhood = [board[(i + ni + height) % height, (j + nj + width) % width]
                                        for ni, nj in neighborhood]
                    
                    # # use this for sum
                    # neighborhood_sum = sum(cell_neighborhood)
                    
                    # use this for average
                    neighborhood_sum = sum(cell_neighborhood)/len(cell_neighborhood)
                    for lower, upper, new_next_state in range_of_neighborhood_sum:
                        if lower <= neighborhood_sum <= upper:
                            next_state = new_next_state
                new_board[i, j] = next_state
        board[:] = new_board[:]
        board_arr.append(board)
    # Calculate Deflate over that board array
    
    try:
        os.remove("board_arrays.txt")
        # print("Removed Successfully")
    except:
        pass
    with open("board_arrays.txt", "w") as output:
        output.write(str(board_arr))
    
    # filedata = open('board_arrays.txt', "rb").readlines()
    # print(filedata)
    filedata = open('board_arrays.txt', 'r', encoding='utf-8').readlines()
    filedata = ''.join(filedata).encode('utf-8')
    # print(filedata)
    compressed = deflate.gzip_compress(filedata, 8)
    deflate_value = compressed.__sizeof__()
    os.system("rm *.txt")
    return deflate_value



# # Testing handlers

# nh_sum1 = [(0.210,0.220,1),(0.350, 0.500, 0), (0.750, 0.850, 0)]
# nh_sum2 = [(0.100,0.280,0),(0.430, 0.550, 1), (0.120, 0.150, 0)]
# range_of_neighborhood_sums = [nh_sum1,nh_sum2]
# print(mnca(range_of_neighborhood_sums))



In [4]:
from tempfile import tempdir
import numpy as np
import sys
import random
import matplotlib.pyplot as plt

np.set_printoptions(threshold=sys.maxsize)

# from mnca import mnca

OVERALL_PROBABILITY_ATLEAST = 3 # works as mutation rate
POPULATION_SIZE = 3 #3
GENERATION_SIZE = 3 #3
NUMBER_OF_NEIGHBORHOODS = 3 
NUMBER_OF_BOUNDS_IN_EACH_NEIGHBORHOOD = 3 

total_sum = 3*NUMBER_OF_NEIGHBORHOODS*NUMBER_OF_BOUNDS_IN_EACH_NEIGHBORHOOD



PROBABILITY_OF_INSERTING_A_NEW_RULE = OVERALL_PROBABILITY_ATLEAST/total_sum
PROBABILITY_OF_REMOVING_A_RULE = OVERALL_PROBABILITY_ATLEAST/total_sum
PROBABILITY_OF_CHANGING_A_RULE = OVERALL_PROBABILITY_ATLEAST/total_sum



def generate_random_rule():
    number_of_neighborhood = NUMBER_OF_NEIGHBORHOODS
    number_of_bounds_in_each_neighborhod_average_or_sum = NUMBER_OF_BOUNDS_IN_EACH_NEIGHBORHOOD
    rule = []
    for i in range(number_of_neighborhood):
        ranges = []
        for j in range(number_of_bounds_in_each_neighborhod_average_or_sum):
            lower = round(random.uniform(0, 1),3)
            upper = round(random.uniform(lower, 1),3)
            new_next_state = random.choice([0, 1]) 
            
            ranges.append((lower, upper, new_next_state))
        rule.append(ranges)
    return rule


def init_population():
    population_size = POPULATION_SIZE
    population = []
    for i in range(population_size):
        individual = generate_random_rule()
        population.append(individual)
    return population



def calculate_deflate(genotype):
    deflate = mnca(genotype)
    return deflate



def roulette_wheel_selection(population, fitness_values):

    sorted_population = [x for _,x in sorted(zip(fitness_values,population))]
    sorted_fitness_scores = [_ for _,x in sorted(zip(fitness_values,population))]
    fitness_values = sorted_fitness_scores
    population = sorted_population
    total_fitness = sum(fitness_values)
    probability_values = [fitness/total_fitness for fitness in fitness_values]
    cumulative_probability = np.cumsum(probability_values)
    selected_index = None
    random_value = random.uniform(0, 1)
    for i in range(len(population)):
        if random_value < cumulative_probability[i]:
            selected_index = i
            break

    return population[selected_index], fitness_values[selected_index]



def mutation(rule):
    try:
        number_of_neighborhoods = len(rule)
        print("►►►► in mutation function rule before mutation: ",str(rule))
        for i in range(number_of_neighborhoods):
            number_of_bounds = len(rule[i])
            for j in range(number_of_bounds):
                prob_insert = PROBABILITY_OF_INSERTING_A_NEW_RULE
                prob_remove = PROBABILITY_OF_REMOVING_A_RULE
                prob_change = PROBABILITY_OF_CHANGING_A_RULE
                random_value = random.uniform(0, 1)
                print("►►►►Probabilities of insert {}, remove {} and change {}.".format(prob_insert, prob_remove, prob_change))
                print("►►►►►►►►We go to insert mode if random value {} is less than {}".format(random_value,prob_insert))
                print("►►►►►►►►We go to remove mode if random value {} is between {} and {}".format(random_value,prob_insert, prob_insert + prob_remove))
                print("►►►►►►►►We go to change mode if random value {} is between {} and {}".format(random_value,prob_insert + prob_remove, prob_insert + prob_remove + prob_change))
                print("►►►►Tossed a coin to check what part of mutation needs to be performed, whether insertion / deletion / updation")
                print("►►►►Value of tossed coin is: ", random_value)
                # Add
                if random_value < prob_insert:
                    print("►►►► In insertion mutation now ►►►►")
                    new_bound = []
                    lower = round(random.uniform(0, 1),3)
                    upper = round(random.uniform(0, 1),3)
                    new_next_state = random.choice([0, 1])
                    if(lower>upper):
                        temp = lower
                        lower = upper
                        upper = temp
                    new_bound.append((lower, upper, new_next_state))
                    print("►►►►rule before insert mutation: ", str(rule))
                    rule[i].extend(new_bound)
                    print("►►►► ►►►► In insertion part of the mutation code ►►►► ►►►►")
                    print("►►►►rule after insert mutation: ", str(rule))
                    print("the bound that has been added is: ",str(new_bound))
                    print("\n")
                # Remove
                elif prob_insert <= random_value < prob_insert + prob_remove:
                    print("►►►► In removing a tuple mutation now ►►►►")
                    if(len(rule[i])>1):
                        print("►►►► Rule before removing tuple mutation: ", str(rule))
                        removed_rule=rule[i].pop(j)
                        number_of_bounds -= 1
                        j -= 1
                        print("►►►► Rule after removing tuple mutation: ", str(rule))
                        print("►►►► tuple removed is: ", removed_rule)
                        print("\n")
                    else:
                        mutation(rule[i])
                # Change
                elif prob_insert + prob_remove <= random_value < prob_insert + prob_remove + prob_change:
                    print("►►►► In change a tuple from the rule by adding a small change mutation part ►►►►")
                    print("►►►► the rule before mutation is: ", str(rule))
                    delta = random.uniform(-0.100,0.100)
                    print("►►►► delta introduced here: ", delta)
                    toss_a_coin = random.uniform(0,1)
                    print("►►►► tossing another coin for tuples' three values, lower limit, upper limit and the next state: ", str(toss_a_coin))
                    if(toss_a_coin<0.33):
                        print("►►►► lower limit is going to be changed:")
                        rule[i][j][0] = rule[i][j][0]+delta
                        if(rule[i][j][0] > 1):
                            rule[i][j][0] = 1
                        elif(rule[i][j][0] < 0):
                            rule[i][j][0] = 0
                        print("►►►► Changed value:")
                        print(rule[i][j][0])
                        print("►►►► New tuple is ")
                        print(rule[i][j])
                        print("►►►► New rule:")
                        print(rule)
                        print("\n")
                    elif(0.33<toss_a_coin<0.66):
                        print("upper limit is going to be changed")
                        rule[i][j][1] = rule[i][j][1]+delta
                        if(rule[i][j][1] > 1):
                            rule[i][j][1] = 1
                        elif(rule[i][j][1] < 0):
                            rule[i][j][1] = 0
                        print("►►►► Changed value:")
                        print(rule[i][j][1])
                        print("►►►► New tuple is ")
                        print(rule[i][j])
                        print("►►►► New rule:")
                        print(rule)
                        print("\n")
                    else:
                        print("state value going to be changed")
                        if(rule[i][j][2] ==0):
                            rule[i][j][2] = 1
                        else:
                            rule[i][j][2] = 0
                        print("►►►► Changed value:")
                        print(rule[i][j][2])
                        print("►►►► New tuple is ")
                        print(rule[i][j])
                        print("►►►► New rule:")
                        print(rule)
                        print("\n")
                    if(rule[i][j][0] > rule[i][j][1]):
                        temp = rule[i][j][0]
                        rule[i][j][0] = rule[i][j][1]
                        rule[i][j][1] = temp
                else:
                    print("►►►►►►►►No change")
                    print("\n")
        print("►►►► ending in mutation function rule after mutation",str(rule))
        print("\n New loop of mutation starts here")

    except:
        pass
    return rule





population = init_population()
generations = GENERATION_SIZE

best_fitness_history = []
average_fitness_history = []
worst_fitness_history = []

for generation in range(generations):
    print("Generation: ", generation)
    
    # Calculate fitness of Individuals
    fitness_values = [calculate_deflate(individual) for individual in population] # Calculate fitness for each individual
    # print("Fitness Values")
    # print(fitness_values)
    # print("old population: ")
    # print(population)
    # print("fitness values ")
    # print(fitness_values)
    # Preserving hisotry
    best_individual_index = np.argmax(fitness_values) # elite index
    # print("best_individual_index")
    # print(best_individual_index)
    import copy
    
    best_individual = copy.deepcopy(population[best_individual_index]) # elite chromosome
    best_fitness = fitness_values[best_individual_index] # elite fitness
    # print("Best Individual: ") # print best chromo
    # print(best_individual)
    # print("Best Fitness: ") # print its fitness
    # print(best_fitness)
    worst_individual_index = np.argmin(fitness_values) # worst history
    worst_fitness = fitness_values[worst_individual_index] # worst fitness

    avg_fitness = np.mean(fitness_values) # average fitness

    best_fitness_history.append(best_fitness) # preserving history
    average_fitness_history.append(avg_fitness) # preserving history
    worst_fitness_history.append(worst_fitness) # preserving history

    
    # Preserving elite
    new_population = [best_individual]  # 1 elite Elitism
    # print("new population ")
    # print(new_population)
    selected_indis = []
    # Selection of the individuals
    while len(new_population) < POPULATION_SIZE:
        select1, _ = roulette_wheel_selection(population, fitness_values)
        # select2, _ = roulette_wheel_selection(population, fitness_values)
        # First deep copy and then mutation
        selected_indis.append(select1)
        deep_copy = copy.deepcopy(select1)
        mutated1 = mutation(deep_copy)
        # mutated2 = mutation(select2)
        # deep_copy = copy.deepcopy(mutated1)
        # new_population.append(deep_copy)
        new_population.append(mutated1)
    
    
    # This function is added as part of debugging 
    def most_frequent(list):
        """
        This function takes a list as input and returns the most frequently occurring element in the list.
        """
        count = 0
        most_freq = list[0]
        
        for element in list:
            freq = list.count(element)
            if freq > count:
                count = freq
                most_freq = element
                
        return most_freq
    
    
    def compare_lists(list1, list2):
        # Get the minimum length of the two lists
        min_length = min(len(list1), len(list2))

        # Loop through each element in the list up to the minimum length
        for a in range(min_length):
            # Get the minimum length of the sub-lists
            sub_min_length = min(len(list1[a]), len(list2[a]))
            # Loop through each element in the sub-list up to the minimum length
            for b in range(sub_min_length):
                # Compare the elements of the two lists
                if list1[a][b] != list2[a][b]:
                    print("►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►")
                    print(f"Element {a+1},{b+1} has changed from {list1[a][b]} to {list2[a][b]}")
                    print("►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►")
                # else:
                #     print("►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►")
                #     print("No change in population")
                #     print("►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►►")

    # return diffs


    
    # print("population after new population")
    # print(population)

    # Questions to answer
    print("QnA Starts now ---------------------------------------------||||||||")
    print("\n")
    print("►►►►What is the init population here?")
    print(population)
    print("\n")
    print("►►►►What is the fitness of population?")
    print(fitness_values)
    print("\n")
    print("►►►►Which one is the best individual?")
    print(best_individual)
    print("\n")
    print("►►►►What is the fitness of the best individual?")
    print(best_fitness)
    print("\n")
    print("►►►►What is the index of the best individual in the population?")
    print(best_individual_index)
    print("\n")
    print("►►►►Which individual is most selected?")
    print(most_frequent(selected_indis))
    print("\n")
    print("►►►►Are best individual and most slected individual same?")
    print(most_frequent(selected_indis) == best_individual)
    print("\n")
    print("►►►►What is the new population?")
    print(new_population)
    print("►►►►What is the difference between old population and new population? Provide a comparartive analysis of the elements that are changed")
    print(compare_lists(population,new_population))
    print("\n")
    print("►►►►Which mutation has worked, was it change in tuple? was it removal of tuple? was it addition of a new tuple?")
    print("TO check that please check mutation log! (above)")
    print("QnA ends now ---------------------------------------------||||||||")
    print("\n")
    print("\n")
    print("\n")
    population = new_population
    # print("population in the end: ",population)
# Plot the graph
# plt.plot(best_fitness_history, label="Best Fitness")
# plt.plot(average_fitness_history, label="Average Fitness")
# plt.plot(worst_fitness_history, label="Worst Fitness")
# plt.xlabel("Generation")
# plt.ylabel("Fitness")
# plt.title("Genetic Algorithm Fitness over Generations")
# plt.legend()
# plt.show()



Generation:  0
►►►► in mutation function rule before mutation:  [[(0.202, 0.602, 1), (0.036, 0.521, 0), (0.72, 0.757, 1)], [(0.544, 0.707, 0), (0.559, 0.763, 1), (0.953, 0.999, 1)], [(0.642, 0.725, 1), (0.354, 0.457, 0), (0.06, 0.65, 0)]]
►►►►Probabilities of insert 0.1111111111111111, remove 0.1111111111111111 and change 0.1111111111111111.
►►►►►►►►We go to insert mode if random value 0.46981130514232217 is less than 0.1111111111111111
►►►►►►►►We go to remove mode if random value 0.46981130514232217 is between 0.1111111111111111 and 0.2222222222222222
►►►►►►►►We go to change mode if random value 0.46981130514232217 is between 0.2222222222222222 and 0.3333333333333333
►►►►Tossed a coin to check what part of mutation needs to be performed, whether insertion / deletion / updation
►►►►Value of tossed coin is:  0.46981130514232217
►►►►►►►►No change


►►►►Probabilities of insert 0.1111111111111111, remove 0.1111111111111111 and change 0.1111111111111111.
►►►►►►►►We go to insert mode if rand